# Interactive PID Control Lab

Welcome to the **Interactive PID Control Lab**! This notebook contains all the hands-on experiments and code examples from the PID control lecture.

## 🎯 Learning Objectives
- **PID controller structure** and transfer function representation
- **Closed-loop analysis** using transfer functions $G_{ry}(s)$
- Effects of **P**, **I**, **D** terms on steady-state error and transient response
- **Disturbance rejection** properties of integral action
- **Practical implementation** considerations and discrete-time forms
- **Manual tuning** strategies based on closed-loop characteristics

---

In [ ]:
# Install required packages
!pip install numpy matplotlib scipy

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import math

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
print("✅ All packages imported successfully!")

# 1. Discrete PID Implementation

Let's start with a practical, discrete-time PID controller that you can use in real projects:

In [ ]:
def pid_discrete(e, st, Kp=1.2, Ki=0.3, Kd=0.05, dt=0.01,
                 tau_d=0.02, umin=-5.0, umax=5.0, Taw=0.1):
    """
    One PID update with filtered D and back-calculation anti-windup.
    st = dict(integ, prev_e, d_filt)
    Returns (u, st)
    """
    integ, prev_e, d_filt = st["integ"], st["prev_e"], st["d_filt"]

    # trapezoidal integral
    integ = integ + 0.5*dt*(e + prev_e)

    # filtered derivative
    alpha = tau_d/(tau_d + dt) if tau_d > 0 else 0.0
    d_raw = (e - prev_e)/dt if dt > 0 else 0.0
    d_filt = alpha*d_filt + (1.0 - alpha)*d_raw

    u_raw = Kp*e + Ki*integ + Kd*d_filt

    # saturation and anti-windup
    u = max(umin, min(umax, u_raw))
    integ += (u - u_raw)/Taw

    st = {"integ": integ, "prev_e": e, "d_filt": d_filt}
    return u, st

print("✅ Discrete PID function defined!")

# 2. First-Order System Simulation

Let's simulate PID control on a simple first-order system: $x_{k+1} = x_k + \Delta t [a (u_k - x_k)]$

In [ ]:
def simulate_pid_first_order(Kp=1.2, Ki=0.3, Kd=0.05, dt=0.01, T=5.0, a=5.0,
                             ref=1.0, tau_d=0.02, umin=-5, umax=5, Taw=0.1):
    n = int(T/dt)
    x = 0.0
    st = {"integ": 0.0, "prev_e": 0.0, "d_filt": 0.0}
    xs, us, es, ts = [], [], [], []

    for k in range(n):
        t = k*dt
        e = ref - x
        u, st = pid_discrete(e, st, Kp, Ki, Kd, dt, tau_d, umin, umax, Taw)
        # plant update
        x = x + dt*(a*(u - x))
        xs.append(x); us.append(u); es.append(e); ts.append(t)
    return np.array(ts), np.array(xs), np.array(us), np.array(es)

# Test the simulation
ts, xs, us, es = simulate_pid_first_order()
print(f"Final output: {xs[-1]:.3f}, Final control: {us[-1]:.3f}")
print("✅ Simulation function ready!")

In [ ]:
# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.plot(ts, xs, label='x (output)', linewidth=2)
ax1.plot(ts, np.ones_like(ts), '--', label='reference', alpha=0.7)
ax1.set_xlabel('time [s]')
ax1.set_ylabel('output')
ax1.set_title('Step response with PID')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(ts, us, label='u (control)', color='orange', linewidth=2)
ax2.set_xlabel('time [s]')
ax2.set_ylabel('control signal')
ax2.set_title('Control effort (with saturation)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Try changing Kp (faster but overshoot), Kd (damps overshoot), Ki (removes bias)")

# 3. Understanding P, I, D Effects on Second-Order Plant

Let's explore how each PID term affects system behavior using a **mass-spring-damper** system. The plant transfer function is:

$$G_p(s) = \frac{\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2}$$

where $\omega_n = 2$ rad/s is the natural frequency and $\zeta = 0.1$ is the damping ratio (lightly damped).

In [ ]:
# Plant: underdamped second-order system
wn = 2.0  # natural frequency
zeta = 0.1  # damping ratio
num_p = [wn**2]
den_p = [1, 2*zeta*wn, wn**2]
Gp = signal.TransferFunction(num_p, den_p)

# Time vector
t = np.linspace(0, 8, 800)

def multiply_tf(tf1, tf2):
    """Multiply two transfer functions manually"""
    # (n1/d1) * (n2/d2) = (n1*n2)/(d1*d2)
    num = np.polymul(tf1.num, tf2.num)
    den = np.polymul(tf1.den, tf2.den)
    return signal.TransferFunction(num, den)

def closed_loop_tf(Gc, Gp):
    """Calculate closed-loop transfer function T = GcGp/(1+GcGp)"""
    # First multiply Gc * Gp
    GcGp = multiply_tf(Gc, Gp)
    
    # T = GcGp / (1 + GcGp)
    # This means T = GcGp.num / (GcGp.den + GcGp.num)
    num_cl = GcGp.num
    den_cl = np.polyadd(GcGp.den, GcGp.num)
    
    return signal.TransferFunction(num_cl, den_cl)

print("✅ Plant and helper functions defined!")
print(f"Plant: {Gp}")

## 3.1 Proportional-Only Control

First, let's see what happens with **P-only** control ($K_i = 0$, $K_d = 0$):

In [ ]:
# Test different Kp values
Kp_values = [0.5, 1.0, 2.0, 4.0]
plt.figure(figsize=(12, 8))

for i, Kp in enumerate(Kp_values):
    # P-only controller
    Gc = signal.TransferFunction([Kp], [1])
    
    # Closed-loop transfer function T(s) = GcGp/(1+GcGp)
    T_cl = closed_loop_tf(Gc, Gp)
    
    # Step response
    t_step, y_step = signal.step(T_cl, T=t)
    
    plt.plot(t_step, y_step, label=f'Kp = {Kp}', linewidth=2)

plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Reference')
plt.xlabel('Time [s]')
plt.ylabel('Output')
plt.title('P-only Control: Effect of Increasing Kp')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 8)
plt.show()

# Show steady-state error for each case
print("Steady-state values (should be 1.0 for perfect tracking):")
for Kp in Kp_values:
    Gc = signal.TransferFunction([Kp], [1])
    T_cl = closed_loop_tf(Gc, Gp)
    _, y_step = signal.step(T_cl, T=t)
    ss_value = y_step[-1]
    ss_error = 1.0 - ss_value
    print(f"Kp = {Kp}: SS value = {ss_value:.3f}, SS error = {ss_error:.3f}")

print("\n🔍 Observations:")
print("• Higher Kp → faster response but more overshoot")
print("• Steady-state error remains (P-only cannot eliminate it for step inputs)")
print("• Too high Kp → system becomes oscillatory/unstable")

## 3.2 Adding Integral Action (PI Control)

Now add integral action to eliminate steady-state error:

In [ ]:
# PI Controller: Gc(s) = Kp + Ki/s
Kp = 1.0  # Fixed proportional gain
Ki_values = [0.0, 0.5, 1.0, 2.0]

plt.figure(figsize=(12, 8))

for Ki in Ki_values:
    # PI controller
    if Ki == 0:
        Gc = signal.TransferFunction([Kp], [1])
        label = f'P-only (Ki = 0)'
    else:
        # Kp + Ki/s = (Kp*s + Ki)/s
        Gc = signal.TransferFunction([Kp, Ki], [1, 0])
        label = f'PI (Ki = {Ki})'
    
    # Closed-loop
    T_cl = closed_loop_tf(Gc, Gp)
    t_step, y_step = signal.step(T_cl, T=t)
    
    plt.plot(t_step, y_step, label=label, linewidth=2)

plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Reference')
plt.xlabel('Time [s]')
plt.ylabel('Output')
plt.title('PI Control: Effect of Adding Integral Action')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 8)
plt.show()

# Check final values
print("Final values with PI control:")
for Ki in Ki_values[1:]:  # Skip Ki=0 case
    Gc = signal.TransferFunction([Kp, Ki], [1, 0])
    T_cl = closed_loop_tf(Gc, Gp)
    _, y_step = signal.step(T_cl, T=t)
    print(f"Ki = {Ki}: Final value = {y_step[-1]:.6f}")

print("\n🔍 Key insight: Integral action eliminates steady-state error but can make the response slower and more oscillatory.")

## 3.3 Adding Derivative Action (Full PID)

Finally, add derivative action to improve transient response:

In [ ]:
# PID Controller: Gc(s) = Kp + Ki/s + Kd*s
Kp, Ki = 1.0, 1.0  # Fixed P and I gains
Kd_values = [0.0, 0.1, 0.2, 0.5]

plt.figure(figsize=(12, 8))

for Kd in Kd_values:
    if Kd == 0:
        # PI only
        Gc = signal.TransferFunction([Kp, Ki], [1, 0])
        label = f'PI (Kd = 0)'
    else:
        # Kp + Ki/s + Kd*s = (Kd*s^2 + Kp*s + Ki)/s
        Gc = signal.TransferFunction([Kd, Kp, Ki], [1, 0])
        label = f'PID (Kd = {Kd})'
    
    T_cl = closed_loop_tf(Gc, Gp)
    t_step, y_step = signal.step(T_cl, T=t)
    
    plt.plot(t_step, y_step, label=label, linewidth=2)

plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Reference')
plt.xlabel('Time [s]')
plt.ylabel('Output')
plt.title('PID Control: Effect of Adding Derivative Action')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 8)
plt.show()

print("\n🔍 Derivative benefits:")
print("• Reduces overshoot and oscillation")
print("• Improves damping and settling time")
print("• Acts like a 'brake' on fast changes")

# 4. Practical PID Class

Here's a complete PID class you can use in your own projects:

In [ ]:
class PID:
    def __init__(self, Kp=1.0, Ki=0.0, Kd=0.0, dt=0.01, tau_d=0.02,
                 umin=-np.inf, umax=np.inf, Taw=0.1):
        self.Kp, self.Ki, self.Kd = Kp, Ki, Kd
        self.dt, self.tau_d = dt, tau_d
        self.umin, self.umax = umin, umax
        self.Taw = Taw
        self.integ = 0.0
        self.prev_e = 0.0
        self.d_filt = 0.0

    def step(self, e):
        dt = self.dt
        # integral (trapezoid)
        self.integ += 0.5*dt*(e + self.prev_e)
        # derivative (filtered)
        if self.tau_d > 0:
            alpha = self.tau_d/(self.tau_d + dt)
            d_raw = (e - self.prev_e)/dt
            self.d_filt = alpha*self.d_filt + (1-alpha)*d_raw
        else:
            self.d_filt = (e - self.prev_e)/dt
        u_raw = self.Kp*e + self.Ki*self.integ + self.Kd*self.d_filt
        # saturation + anti-windup
        u = max(self.umin, min(self.umax, u_raw))
        self.integ += (u - u_raw)/self.Taw
        self.prev_e = e
        return u
    
    def reset(self):
        """Reset integrator and derivative filter"""
        self.integ = 0.0
        self.prev_e = 0.0
        self.d_filt = 0.0

# Test the PID class
pid = PID(Kp=1.2, Ki=0.3, Kd=0.05)
print("✅ PID class ready to use!")
print(f"Test: error=1.0 → control = {pid.step(1.0):.3f}")
print(f"Test: error=0.5 → control = {pid.step(0.5):.3f}")

# 5. Interactive PID Tuning Exercise

Try different PID parameter combinations and observe their effects:

In [ ]:
# Interactive PID tuning exercise
def plot_pid_response(Kp=1.0, Ki=0.5, Kd=0.1):
    """Plot step response for given PID parameters"""
    
    # Create PID controller
    if Ki == 0 and Kd == 0:
        Gc = signal.TransferFunction([Kp], [1])
        controller_name = f"P-only (Kp={Kp})"
    elif Kd == 0:
        Gc = signal.TransferFunction([Kp, Ki], [1, 0])
        controller_name = f"PI (Kp={Kp}, Ki={Ki})"
    else:
        Gc = signal.TransferFunction([Kd, Kp, Ki], [1, 0])
        controller_name = f"PID (Kp={Kp}, Ki={Ki}, Kd={Kd})"
    
    # Closed-loop response
    T_cl = closed_loop_tf(Gc, Gp)
    t_step, y_step = signal.step(T_cl, T=t)
    
    # Calculate performance metrics
    final_value = y_step[-1]
    steady_state_error = abs(1.0 - final_value)
    peak_value = np.max(y_step)
    overshoot = (peak_value - 1.0) * 100 if peak_value > 1.0 else 0
    
    # Find settling time (2% criterion)
    settle_band = 0.02
    settled_indices = np.where(np.abs(y_step - final_value) <= settle_band)[0]
    settle_time = t[settled_indices[0]] if len(settled_indices) > 0 else t[-1]
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(t_step, y_step, linewidth=3, label=controller_name)
    plt.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Reference')
    plt.axhline(y=1+settle_band, color='r', linestyle=':', alpha=0.5, label='±2% band')
    plt.axhline(y=1-settle_band, color='r', linestyle=':', alpha=0.5)
    
    plt.xlabel('Time [s]')
    plt.ylabel('Output')
    plt.title(f'Step Response - {controller_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 8)
    
    # Add performance text
    perf_text = f"""Performance Metrics:
• Steady-state error: {steady_state_error:.4f}
• Overshoot: {overshoot:.1f}%
• Settling time: {settle_time:.2f}s
• Final value: {final_value:.4f}"""
    
    plt.text(0.02, 0.98, perf_text, transform=plt.gca().transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return steady_state_error, overshoot, settle_time

print("🎛️ Interactive PID Tuning Exercise")
print("Try different combinations and observe the effects:")
print()

# Example: Try these combinations
tuning_examples = [
    ("P-only (low gain)", 0.5, 0.0, 0.0),
    ("P-only (high gain)", 3.0, 0.0, 0.0),
    ("PI control", 1.0, 0.8, 0.0),
    ("PID control", 1.0, 0.8, 0.15)
]

for name, kp, ki, kd in tuning_examples:
    print(f"\n=== {name} ===")
    sse, os, st = plot_pid_response(kp, ki, kd)

# 🎯 Conclusion

Congratulations! You've completed the Interactive PID Control Lab. Here's what you've learned:

## Key Takeaways

1. **P-control**: Fast response but with steady-state error
2. **I-control**: Eliminates steady-state error but can cause oscillation
3. **D-control**: Improves damping and reduces overshoot
4. **Practical implementation**: Filtering, anti-windup, and saturation are essential
5. **Tuning strategy**: Start with P, add D for damping, then I for accuracy

## Next Steps

- Try the PID class on your own control problems
- Experiment with different plant models
- Learn about advanced topics like feedforward control and gain scheduling
- Explore modern control methods (LQR, MPC, H∞)

## Resources

- **Textbook**: Åström & Murray - "Feedback Systems"
- **Online**: Control tutorials from University of Michigan
- **Software**: MATLAB Control Toolbox, Python Control Library

---

**Happy controlling! 🎛️**